### Khai báo thư viện


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, posexplode, struct, expr, collect_list, array_sort

### Khởi tạo SparkSession


In [26]:
spark = SparkSession.builder \
    .appName("Polygon Overlap Detection Structured") \
    .master("local[*]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

### Đọc dữ liệu


In [27]:
df = spark.read.parquet("shapes.parquet")

### Chuẩn hoá định dạng


In [28]:
exploded = df.select(
    col("shape_id"),
    posexplode("vertices").alias("index", "point")
)

### Tách x, y từ điểm


In [29]:
df_points = exploded.select(
    "shape_id", "index",
    col("point")[0].alias("x"),
    col("point")[1].alias("y")
)

### Sắp xếp lại các điểm x, y


In [30]:
sorted_points = df_points.groupBy("shape_id") \
    .agg(array_sort(collect_list(struct("x", "y"))).alias("sorted_points"))

### Tính hộp bao cho mỗi hình


In [31]:
bounding_boxes = df_points.groupBy("shape_id").agg(
    expr("min(x)").alias("min_x"),
    expr("max(x)").alias("max_x"),
    expr("min(y)").alias("min_y"),
    expr("max(y)").alias("max_y")
)

### Ghép các hình theo thứ tự


In [32]:

bbox_a = bounding_boxes.alias("a")
bbox_b = bounding_boxes.alias("b")
pairs = bbox_a.crossJoin(bbox_b) \
    .filter(expr("a.shape_id < b.shape_id"))

### Kiểm tra chồng lắp bằng hộp bao


In [33]:
pairs_overlap = pairs.filter(
    (col("a.max_x") > col("b.min_x")) &
    (col("a.min_x") < col("b.max_x")) &
    (col("a.max_y") > col("b.min_y")) &
    (col("a.min_y") < col("b.max_y"))
)

### Xuất kết quả


In [34]:
result_df = pairs_overlap.select(col("a.shape_id").alias("shape_1"), col("b.shape_id").alias("shape_2"))

### Sắp xếp theo thứ tự


In [35]:
result_df = result_df.withColumn("s1_num", expr("int(split(shape_1, '_')[1])")) \
                     .withColumn("s2_num", expr("int(split(shape_2, '_')[1])")) \
                     .orderBy("s1_num", "s2_num") \
                     .select("shape_1", "shape_2")

### Lưu ra CSV không có header


In [36]:
result_df.write.mode("overwrite").csv("output_folder", header=False)